In [2]:
import numpy as np

In [5]:
utf8_repr = {
    "A": 0b01000001,
    "C": 0b01000011,
    "T": 0b01010100,
    "G": 0b01000111
}

def encode_dna_bin(bin_seq: np.ndarray):
    # split the sequence into 8-bit chunks
    chunks = np.array(np.split(bin_seq, bin_seq.size // 8), dtype=object)
    return chunks

In [37]:
def get_binary_repr(seq: str):
    seq = np.array([utf8_repr[c] for c in seq], dtype=np.uint8)

    # each element is an 8-bit integer, so we need to convert it to a binary string
    binary_repr = np.array([np.binary_repr(i, width=8) for i in seq], dtype=object)

    # convert the binary string to a numpy array of integers
    binary_repr = np.array([[int(c) for c in s] for s in binary_repr], dtype=np.uint8)

    return binary_repr

In [39]:
get_binary_repr("ACTG")

array([[0, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 1, 1],
       [0, 1, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 1]], dtype=uint8)

In [38]:
# "ACG" -> [0b01000001, 0b01000011, 0b01000111]

print(get_binary_repr("ACG"))
seq = 0b010000010100001101000011
seq_str = format(seq, 'b')

if len(seq_str) % 8 != 0:
    seq_str = "0" * (8 - len(seq_str) % 8) + seq_str

bin_seq = np.array([int(x) for x in str(seq_str)])

chunks = encode_dna_bin(bin_seq)

[[0 1 0 0 0 0 0 1]
 [0 1 0 0 0 0 1 1]
 [0 1 0 0 0 1 1 1]]
